In [106]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# 1. Test function
今回最適化する関数を定義する。

In [121]:
x

array([-1.        , -0.97979798, -0.95959596, -0.93939394, -0.91919192,
       -0.8989899 , -0.87878788, -0.85858586, -0.83838384, -0.81818182,
       -0.7979798 , -0.77777778, -0.75757576, -0.73737374, -0.71717172,
       -0.6969697 , -0.67676768, -0.65656566, -0.63636364, -0.61616162,
       -0.5959596 , -0.57575758, -0.55555556, -0.53535354, -0.51515152,
       -0.49494949, -0.47474747, -0.45454545, -0.43434343, -0.41414141,
       -0.39393939, -0.37373737, -0.35353535, -0.33333333, -0.31313131,
       -0.29292929, -0.27272727, -0.25252525, -0.23232323, -0.21212121,
       -0.19191919, -0.17171717, -0.15151515, -0.13131313, -0.11111111,
       -0.09090909, -0.07070707, -0.05050505, -0.03030303, -0.01010101,
        0.01010101,  0.03030303,  0.05050505,  0.07070707,  0.09090909,
        0.11111111,  0.13131313,  0.15151515,  0.17171717,  0.19191919,
        0.21212121,  0.23232323,  0.25252525,  0.27272727,  0.29292929,
        0.31313131,  0.33333333,  0.35353535,  0.37373737,  0.39

In [124]:
data

array([-1.        , -0.97979798, -0.95959596, -0.93939394, -0.91919192,
       -0.8989899 , -0.87878788, -0.85858586, -0.83838384, -0.81818182,
       -0.7979798 , -0.77777778, -0.75757576, -0.73737374, -0.71717172,
       -0.6969697 , -0.67676768, -0.65656566, -0.63636364, -0.61616162,
       -0.5959596 , -0.57575758, -0.55555556, -0.53535354, -0.51515152,
       -0.49494949, -0.47474747, -0.45454545, -0.43434343, -0.41414141,
       -0.39393939, -0.37373737, -0.35353535, -0.33333333, -0.31313131,
       -0.29292929, -0.27272727, -0.25252525, -0.23232323, -0.21212121,
       -0.19191919, -0.17171717, -0.15151515, -0.13131313, -0.11111111,
       -0.09090909, -0.07070707, -0.05050505, -0.03030303, -0.01010101,
        0.01010101,  0.03030303,  0.05050505,  0.07070707,  0.09090909,
        0.11111111,  0.13131313,  0.15151515,  0.17171717,  0.19191919,
        0.21212121,  0.23232323,  0.25252525,  0.27272727,  0.29292929,
        0.31313131,  0.33333333,  0.35353535,  0.37373737,  0.39

In [126]:
def test_func(x,y):
    xo=0.2
    yo=0.6
    return np.exp(-((x-xo)**2+(y-yo)**2))*np.cos(2*np.pi*(x**2+2*y**2))
test_func = np.vectorize(test_func)

def gen_xy(etax, etay):
    # recieve normalized coordinates [0-1]
    upper=1
    lower=-1
    x=(etax-0.5)*(upper-lower)
    x=x.reshape(-1,1)
    y=(etay-0.5)*(upper-lower)
    y=y.reshape(-1,1)
    return np.hstack((x,y))

etax = np.linspace(0,1,100)
etay = np.linspace(0,1,100)

data = gen_xy(etax,etay)
x=data[:,0]
y=data[:,1]
xx, yy = np.meshgrid(x,y)

z = test_func(xx.ravel(), yy.ravel())
z = np.reshape(z,np.shape(xx))



## Visualize
fig = go.Figure(data=[go.Surface(z=z,x=x,y=y)])
#fig = go.Figure(data=[go.Mesh3d(z=z,x=x,y=y,color="black")])
fig.show()



# 2. Initial sample
初期サンプルを複数計算する。

In [128]:
data_eta = np.random.random_sample(size=(10,2))
data = gen_xy(data_eta[:,0],data_eta[:,1])
z = test_func(data[:,0],data[:,1])
data = np.hstack((data,z.reshape(-1,1)))
#data =  np.hstack((data,test_func(data[:,0],data[:,1])))

fig.add_trace(go.Scatter3d(x=data[:,0], y=data[:,1], z=data[:,2], 
                            mode="markers",
                            marker=dict(
                                color='LightSkyBlue',
                                size=3,
                                line=dict(
                                    color='MediumPurple',
                                    width=1
                                )
                            )
                        ))
fig.show()

# 2. サンプル点から関数を予測

## 2.1 ガウス過程

In [29]:
np.c_[xx.ravel(), yy.ravel()][0]

array([2. , 4.3])

In [25]:
#https://qiita.com/Ringa_hyj/items/e42c726cdee25767105d

import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.svm import SVR

df = px.data.iris()

#Xからの写像を作っておく

margin = 0
X = df[['sepal_width', 'sepal_length']]
y = df['petal_width']
model = SVR(C=1.)
model.fit(X, y)

#細かい点(メッシュ,グリッド)を発生

mesh_size = .02
x_min, x_max = X.sepal_width.min() - margin, X.sepal_width.max() + margin
y_min, y_max = X.sepal_length.min() - margin, X.sepal_length.max() + margin
xrange = np.arange(x_min, x_max, mesh_size)
yrange = np.arange(y_min, y_max, mesh_size)
xx, yy = np.meshgrid(xrange, yrange)
#メッシュのすべての点について予測

pred = model.predict(np.c_[xx.ravel(), yy.ravel()])
pred = pred.reshape(xx.shape)

#元の点をplotしてから、x1,x2によるgrid面をzによって押し上げる
#面は全ての点をつなぐsurfaceをつかって描く

fig = px.scatter_3d(df, x='sepal_width', y='sepal_length', z='petal_width')
fig.update_traces(marker=dict(size=5))
fig.add_traces(go.Surface(x=xrange, y=yrange, z=pred, name='pred_surface'))
fig.show()

NameError: name 'f' is not defined

In [9]:
xx

array([[2.  , 2.02, 2.04, ..., 4.36, 4.38, 4.4 ],
       [2.  , 2.02, 2.04, ..., 4.36, 4.38, 4.4 ],
       [2.  , 2.02, 2.04, ..., 4.36, 4.38, 4.4 ],
       ...,
       [2.  , 2.02, 2.04, ..., 4.36, 4.38, 4.4 ],
       [2.  , 2.02, 2.04, ..., 4.36, 4.38, 4.4 ],
       [2.  , 2.02, 2.04, ..., 4.36, 4.38, 4.4 ]])

In [12]:
len(xx.ravel())

21901

In [18]:
xx.ravel()

array([2.  , 2.02, 2.04, ..., 4.36, 4.38, 4.4 ])

In [22]:
yy.ravel()

array([4.3, 4.3, 4.3, ..., 7.9, 7.9, 7.9])

In [24]:
np.shape(xx)

(181, 121)